In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch8
!touch ch8/__init__.py

mkdir: cannot create directory ‘ch8’: File exists


In [0]:
import imp

## 8.1 变量不是盒子

- 人们经常使用“变量是盒子”这样的比喻，但是这有碍于理解面向对象语言中的引用式变量
- Python 变量为引用式变量，因此最好把它们理解为附加在对象上的标注

###### 示例 8-1　变量 a 和 b 引用同一个列表，而不是那个列表的副本

In [0]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200407155614.png width=600>

###### 示例 8-2 创建对象之后才会把变量分配给对象

In [0]:
class Gizmo:
  def __init__(self):
    print("Gizmo id: %d" % id(self))

In [2]:
x = Gizmo()

Gizmo id: 139674107269752


In [3]:
y = Gizmo() * 10  # 将 gizmo 用于乘法运算会出错，但是会先创建对象

Gizmo id: 139674107270144


TypeError: ignored

In [4]:
", ".join([item for item in dir() if not item.startswith("_")])  # 不会创建变量 y，因为在对赋值语句的右边进行求值时抛出了异常

'Gizmo, In, Out, exit, get_ipython, quit, x'

## 8.2 标识、相等性和别名

- 每个变量都有标识、类型和值
- 对象一旦创建，它的标识绝不会变；可以把标识理解为对象在内存中的地址
- `is` 运算符比较两个对象的标识
- `id()` 函数返回对象标识的整数表示
  - 对象 ID 的真正意义在不同的实现中有所不同
    - 在 CPython 中，`id()` 返回对象的内存地址
    - 但在其他 Python 解释器中可能是别的值
  - 关键是，ID 一定是唯一的数值标注，而且在对象的生命周期中绝不会变

###### 示例 8-3　charles 和 lewis 指代同一个对象

In [0]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles  # lewis 是 charles 的别名
lewis is charles

True

In [0]:
id(charles), id(lewis)

(140169600493104, 140169600493104)

###### 示例 8-4　alex 与 charles 比较的结果是相等，但 alex 不是 charles

In [0]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832}
alex == charles  # 比较两个对象，结果相等。背后调用的是 dict 的 __eq__ 方法

True

In [0]:
alex is not charles 

True

- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408080702.png width=600>

### 8.2.1 在 `==` 和 `is` 之间选择

- `==` 运算符比较两个对象的值（对象中保存的数据）
- is 比较对象的标识
- 通常，我们关注的是值，而不是标识，因此 Python 代码中 `==` 出现的频率比 `is` 高

***
- 在变量和单例值之间比较时，应该使用 is
- 最常使用 is检查变量绑定的值是不是 None

In [0]:
x = None
x is None

True

In [0]:
x = 1
x is not None

True

***
- `is` 运算符比 `==` 速度快
  - 因为 `is` 不能重载，所以 Python 不用寻找并调用特殊方法，而是直接比较两个整数 ID
  - 而 `a == b` 是语法糖，等同于`a.__eq__(b)`
    - 继承自 object 的 `__eq__` 方法比较两个对象的 ID，结果与 `is` 一样
    - 但是多数内置类型使用更有意义的方式覆盖了 `__eq__`方法，会考虑对象属性的值
    - 相等性测试可能涉及大量处理工作，例如，比较大型集合或嵌套层级深的结构时

### 8.2.2 元组的相对不可变性

- 元组与多数 Python 集合（列表、字典、集，等等）一样，保存的是对象的引用
  - 如果引用的元素是可变的，即便元组本身不可变，元素依然可变。
  - 即元组的不可变性其实是指 `tuple` 数据结构的物理内1容（即保存的引用）不可变，与引用的对象无关
  - 由于元组的此特性，因此有些元组不可散列
- `str`、`bytes` 和 `array.array` 等单一类型序列是扁平的，它们保存的不是引用，而是在连续的内存中保存数据本身（字符、字节和数字）

###### 示例 8-5　一开始，t1 和 t2 相等，但是修改 t1 中的一个可变元素后，二者不相等了

In [0]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2  # t1 与 t2 是不同的对象，但是两者相等

True

In [0]:
id(t1[-1])

139940199905480

In [0]:
t1[-1].append(99)

In [0]:
t1

(1, 2, [30, 40, 99])

In [0]:
id(t1[-1])  # t1[-1] 的标识没变，但值发生了改变

139940199905480

In [0]:
t1 == t2

False

## 8.3 默认作浅复制

- 复制列表（或多数内置的可变集合）最简单的方式是使用内置的类型构造方法

In [0]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [0]:
l1 == l2

True

In [0]:
l1 is l2

False

- 对列表和其他可变序列来说，还能使用简洁的 `l3 = l1[:]` 语句创建副本

In [0]:
l3 = l1[:]
l3

[3, [55, 44], (7, 8, 9)]

- 构造方法或 `[:]` 做的是浅复制（即复制了最外层容器，副本中的元素是源容器中元素的引用）
  - 如果所有元素都是不可变的，那么这样没有问题，还能节省内存
  - 但是，如果有可变的元素，可能就会导致意想不到的问题

###### 示例 8-6　为一个包含另一个列表的列表做浅复制；把这段代码复制粘贴到 Python Tutor 网站中，要查看动画效果

In [0]:
l1  = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)  # 1
l1.append(100)  # 2
l1[1].remove(55)  # 3
print("l1", l1)
print("l2", l2)
l2[1] += [33, 22]  # 4
l2[2] += (10, 11)  # 5
print('l1:', l1)
print('l2:', l2)

l1 [3, [66, 44], (7, 8, 9), 100]
l2 [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


1. `l2` 是 `l1` 的浅复制
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408102255.png width=500>



2. 将 100 追加到 `l1` 中， 对 `l2` 没有影响
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408102509.png width=500>

3. 把内部列表 `l1[1]` 中的 55 删除。这对 `l2` 有影响，因为 `l2[1]` 绑定的列表与 `l1[1]` 是同一个
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408102859.png width=500>

4. 对可变的对象来说，如 `l2[1]` 引用的列表，`+=` 运算符就地修改列表。这次修改在 `l1[1]` 中也有体现，因为它是 `l2[1]` 的别名
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408103121.png width=500>

5. 对元组来说，+= 运算符创建一个新元组，然后重新绑定给变量l2[2]。这等同于 l2[2] = l2[2] + (10, 11)。现在，l1 和 l2 中最后位置上的元组不是同一个对象
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200408103322.png width=500>

### 为任意对象做深复制和浅复制

- `copy` 模块提供的 `deepcopy` 和 `copy` 函数能为任意对象做深复制（即副本不共享内部对象的引用）和浅复制

###### 示例 8-8　校车乘客在途中上车和下车

In [0]:
%%writefile ch8/bus.py

class Bus:

  def __init__(self, passengers = None):
    if passengers is None:
      self.passengers =  []
    else:
      self.passengers = list(passengers)

  def pick(self, name):
    self.passengers.append(name)

  def drop(self, name):
    self.passengers.remove(name)

Overwriting ch8/bus.py


In [0]:
import ch8.bus
imp.reload(ch8.bus)
from ch8.bus import Bus


In [0]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(139940180405160, 139940180405272, 139940180405216)

In [0]:
bus1.drop('Bill')
bus2.passengers  # bus1 中的 bill 下车后， bus2 中也没有他了

['Alice', 'Claire', 'David']

In [0]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)  # 因为 bus2 是 bus1 的浅复制版本，因此两都共享一个列表对象

(139940180388424, 139940180388424, 139940180512456)

In [0]:
bus3.passengers  # bus3 是 bus1 的深复制副本，因此其 passengers 属性指向另外一个列表

['Alice', 'Bill', 'Claire', 'David']

#### 循环引用

- 深复制不是件简单的事。如果对象有循环引用，那么这个朴素的算法会进入无限循环
- `deepcopy` 函数会记住已经复制的对象，因此能优雅地处理循环引用
- 深复制有时可能太深了。例如，对象可能会引用不该复制的外部资源或单例值
  - 可以实现特殊方法 `__copy__() `和`__deepcopy__()`，控制`copy`和`deepcopy`的行为，

###### 示例 8-10　循环引用：`b` 引用 `a`，然后追加到 `a` 中；`deepcopy` 会想办法复制 `a`

In [0]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [0]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

## 8.4 函数的参数作为引用时

- Python 唯一支持的参数传递模式是共享传参（call by sharing）
- 共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是说，函数内部的形参是实参的别名
- 函数可能会修改作为参数传入的可变对象，但是无法修改那些对象的标识（即不能把一个对象替换成另一个对象）

###### 示例 8-11　函数可能会修改接收到的任何可变对象

In [0]:
def f(a, b):
  a += b
  return a

In [0]:
x, y = 1, 2
f(x, y)

3

In [0]:
x, y  # 数字 x 没变

(1, 2)

In [0]:
a, b = [1, 2], [3, 4]
f(a, b)

[1, 2, 3, 4]

In [0]:
a, b  # 列表 a 发生了改变

([1, 2, 3, 4], [3, 4])

In [0]:
t, u = (10, 20), (30, 40)
f(t, u)

(10, 20, 30, 40)

In [0]:
t, u  # 元组 t 没有发生改变

((10, 20), (30, 40))

### 8.4.1 不要使用可变类型作为参数的默认值

- 默认值在定义函数时计算（通 常在加载模块时），因此默认值变成了函数对象的属性。
- 如果默认值是可变对象，而且修改了它的值，那么后续的函数调用都会受到影响
- 由于可变默认值存在的这个问题，通常使用 `None` 作为接收可变值的参数的默认值

###### 示例 8-12　一个简单的类，说明可变默认值的危险

- 没有指定初始乘客的 `HauntedBus` 实例会共享同一个乘客列表

In [0]:
%%writefile ch8/hauntedbus.py
class HauntedBus:
  """备受幽灵乘客折磨的校车"""

  def __init__(self, passengers=[]):
    self.passengers = passengers

  def pick(self, name):
    self.passengers.append(name)

  def drop(self, name):
    self.passengers.remove(name)

Writing ch8/hauntedbus.py


In [0]:
import ch8.hauntedbus
imp.reload(ch8.hauntedbus)
from ch8.hauntedbus import HauntedBus

In [0]:
bus1 = HauntedBus(['Alice', 'Bill'])  # 不使用默认值，不会出现问题
bus1.passengers

['Alice', 'Bill']

In [0]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [0]:
bus2 = HauntedBus()  # 使用默认值
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [0]:
bus3 = HauntedBus()  # 继续使用默认值
bus3.passengers  # 默认的列表此时不为空

['Carrie']

In [0]:
bus3.pick('Dive')  # 修改 bus3 的乘客，会影响 bus2
bus2.passengers

['Carrie', 'Dive']

In [0]:
bus2.passengers is bus3.passengers  # bus2.passengers 与 bus3.passengers 指向的是同一列表

True

In [0]:
bus1.passengers  # bus1.passengers 是不同的列表

['Bill', 'Charlie']

***
- 可以在 `HauntedBus.__init__ `对象的 `__defaults__`属性中查看有哪些幽灵学生

In [0]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [0]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dive'],)

- 可以验证 `bus2.passengers `是一个别名，它绑定到`HauntedBus.__init__.__defaults__ `属性的第一个元素上

In [0]:
bus2.passengers is HauntedBus.__init__.__defaults__[0]

True

### 8.4.2 防御可变参数

- 如果定义的函数接收可变参数，应该谨慎考虑调用方是否期望修改传入的参数
  - 例如如果函数接收一个字典，而且在处理的过程中要修改它，那么这个副作用要不要体现到函数外部？
  - 具体情况应具体分析。需要函数的编写者和调用方达成共识

###### 示例 8-15　一个简单的类，说明接受可变参数的风险

In [0]:
%%writefile ch8/twilightbus.py
class TwilightBus:
  """让乘客销声匿迹的校车"""

  def __init__(self, passengers=None):
    self.passengers = [] if passengers is None else passengers

  def pick(self, name):
    self.passengers.append(name)

  def drop(self, name):
    self.passengers.remove(name)

Overwriting ch8/twilightbus.py


In [0]:
import ch8.twilightbus
imp.reload(ch8.twilightbus)
from ch8.twilightbus import TwilightBus

In [0]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)  # 从 TwilightBus 下车后，乘客消失了
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

- 此处正确的做法应该是校车自己维护乘客列表
- ```python
  def __init__(self, passengers=None):
    self.passengers = [] if passengers is None else list(passengers)
```
- 在内部像这样处理乘客列表，就不会影响初始化校车时传入的参数
  - 此外，这种处理方式还更灵活：即传给 `passengers` 参数的值可以是元组或任何其他可迭代对象
    - 例如 `set` 对象，甚至数据库查询结果，因为 `list` 构造方法接受任何可迭代对象

## 8.5 `del` 和垃圾回收

- `del` 语句删除名称，而不是对象
- `del` 命令可能会导致对象被当作垃圾回收，但是仅当删除的变量保存的是对象的最后一个引用，或者无法得到对象时。
- 重新绑定也可能会导致对象的引用数量归零，导致对象被销毁
***
- `__del__` 特殊方法不会销毁实例，不应该在代码中调用
- 即将销毁实例时，Python 解释器会调用 `__del__` 方法，给实例最后的机会，释放外部资源
- 自己编写的代码很少需要实现 `__del__` 代码，有些 Python 新手会花时间实现，但却吃力不讨好，因为 `__del__` 很难用对
***
- 在 CPython 中，垃圾回收使用的主要算法是引用计数
  - 每个对象都会统计有多少引用指向自己。当引用计数归零时，对象立即就被销毁
    - CPython 会在对象上调用 `__del__` 方法（如果定义了），然后释放分配给对象的内存。
- CPython　2.0 增加了分代垃圾回收算法，用于检测引用循环中涉及的对象组——如果一组对象之间全是相互引用，即使再出色的引用方式也会导致组中的对象不可获取
- Python 的其他实现有更复杂的垃圾回收程序，而且不依赖引用计数，这意味着，对象的引用数量为零时可能不会立即调用 `__del__` 方法。

###### 示例 8-16　没有指向对象的引用时，监视对象生命结束时的情形

In [0]:
import weakref
s1 = {1, 2, 3}
s2 = s1


def bye():
  print('Gone  with the wind...')


ender = weakref.finalize(s1, bye)
ender.alive

Gone  with the wind...


True

In [0]:
del s1  # del 不删除对象，而是删除对象的引用
ender.alive

True

In [0]:
s2 = 'spam'  # 重新绑定最后一个引用 s2，让 {1, 2, 3} 无法获取。对象被销毁了，调用了 bye 回调，ender.alive 的值变成了 False

In [0]:
ender.alive

False

## 8.6 弱引用

- 正是因为有引用，对象才会在内存中存在
  - 当对象的引用数量归零后，垃圾回收程序会把对象销毁
  - 但是，有时需要引用对象，而不让对象存在的时间超过所需时间，这经常用在缓存中
- 弱引用不会增加对象的引用数量
  - 引用的目标对象称为所指对象（referent）
  - 因此我们说，弱引用不会妨碍所指对象被当作垃圾回收
  - 弱引用在缓存应用中很有用，因为不想仅因为被缓存引用着而始终保存缓存对象

###### 示例 `8`-`17`　弱引用是可调用的对象，返回的是被引用的对象；如果所指对象不存在了，返回 `None`

- Python 控制台会自动把 `_` 变量绑定到结果不为 `None` 的表达式结果上

In [0]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)  # 创建弱引用对象
wref

<weakref at 0x7f465cbaa6d8; to 'set' at 0x7f465cc084a8>

In [0]:
wref()  # 调用 wref() 返回的是被引用的对象，{0, 1}。因为这是控制台会话，所以 {0, 1} 会绑定给 _ 变量。

{0, 1}

In [0]:
a_set = {2, 3, 4}  # a_set 不再指代 {0, 1} 集合，因此集合的引用数量减少了。但是 _ 变量仍然指代它

In [0]:
wref()  #

{0, 1}

In [0]:
wref is None  # 计算这个表达式时，{0, 1} 存在，因此 wref() 不是 None。但是，随后 _ 绑定到结果值 False。现在 {0, 1} 没有强引用了

False

In [0]:
wref is None  # {1, 2} 不存在， wref 为 None

False

### 8.6.1 `WeakValueDictionary` 简介

- `WeakValueDictionary` 类实现的是一种可变映射，里面的值是对象的弱引用
- 被引用的对象在程序中的其他地方被当作垃圾回收后，对应的键会自动从 `WeakValueDictionary` 中删除
- 因此，`WeakValueDictionary` 经常用于缓存

###### 示例 8-18　`Cheese` 有个 `kind` 属性和标准的字符串表示形式

In [0]:
%%writefile ch8/cheese.py
class Cheese:

  def __init__(self, kind):
    self.kind = kind

  def __repr__(self):
    return 'Cheese(%r)' % self.kind

Writing ch8/cheese.py


###### 示例 8-19　顾客：“你们店里到底有没有奶酪？”

In [0]:
import ch8.cheese
imp.reload(ch8.cheese)
from ch8.cheese import Cheese

In [0]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
  stock[cheese.kind] = cheese

sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [0]:
del catalog
sorted(stock.keys())  # for 循环中的 cheese 是全局变量，除非被显示删除，否则会一直存在，因此 stock 中最后只剩下 Parmesam

['Parmesan']

In [0]:
del cheese
sorted(stock.keys())

[]

#### `WeakKeyDictionary()`

- `WeakKeyDictionary` 的键是弱引用，可以为应用中其他部分拥有的对象附加数据
  - 这样就无需为对象添加属性
  - 对覆盖属性访问权限的对象尤其有用。

#### `WeakSet` 类

- 保存元素弱引用的集合类。元素没有强引用时，集合会把它删除。”
- 如果一个类需要知道所有实例，一种好的方案是创建一个 `WeakSet` 类型的类属性，保存实例的引用
- 如果使用常规的 `set`，实例永远不会被垃圾回收，因为类中有实例的强引用，而类存在的时间与 Python 进程一样长，除非显式删除类

### 8.6.2　弱引用的局限

- 不是每个 Python 对象都可以作为弱引用的目标（或称所指对象）。基本的 `list` 和 `dict` 实例不能作为所指对象，但是它们的子类可以解决这个问题
- `int` 和 `tuple` 实例不能作为弱引用的目标，它们的子类也不行
- 这些局限基本上是 CPython 的实现细节，在其他 Python 解释器中情况可能不一样。这些局限是内部优化导致的结果。

In [0]:
class MyList(list):
  """list 的子类，实例可以作为弱引用目标"""

In [0]:
a_list = MyList(range(10))

wref_to_a_list = weakref.ref(a_list)

## 8.7 Python对不可变类型施加的把戏

- 对元组 `t` 来说，`t[:]` 不创建副本，而是返回同一个对象的引用。此外，`tuple(t)` 获得的也是同一个元组的引用
- `str`、`bytes` 和 `frozenset` 实例也有这种行为
  - 注意，`frozenset` 实例不是序列，因此不能使用 `fs`[:]（`fs` 是一个 `frozenset` 实例）
  - 但是，`fs.copy()` 具有相同的效果
    - 它会返回同一个对象的引用，而不是创建一个副本

###### 示例 8-20　使用另一个元组构建元组，得到的其实是同一个元组

In [0]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [0]:
t3 = t1[:]
t3 is t1

True

##### 共享字符串
- 共享字符串字面量是一种优化措施，称为驻留（interning）
  - CPython 还会在小的整数上使用这个优化措施，防止重复创建“热门”数字，如0、-1 和 42
  - CPython 不会驻留所有字符串和整数，驻留的条件是实现细节，而且没有文档说明
- 由于驻留特性，比较字符串或整数是否相等时，应该使用 `==`，而不是 `is`

###### 示例 8-21　字符串字面量可能会创建共享的对象

In [0]:
t1 = (1, 2, 3)
t2 = (1, 2, 3)
t2 is t1

False

In [0]:
s1 = 'ABC'
s2 = 'ABC'
s1 is s2

True

##### 不会对实际使用造成影响
- 本节中的例子，包括 frozenset.copy() 的行为，是“善意的谎言”，能节省内存，提升解释器的速度
- 但它们不会为编程带来任何麻烦，因为只有不可变类型会受到影响